In [1]:
#https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-dataset
!rm -rf sample_data
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
!echo "Data downloaded ⬇️"
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12_2.zip
!echo "Annotations downloaded ⤵️"

!tar zxf mpii_human_pose_v1.tar.gz
!echo "Untared 🤐"

!unzip mpii_human_pose_v1_u12_2.zip
!echo "Unziped 🤐"

!echo "Data ready ✅"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  doc-base
The following NEW packages will be installed:
  pv tree
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 92.4 kB of archives.
After this operation, 255 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pv amd64 1.6.6-1build2 [44.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 92.4 kB in 1s (103 kB/s)
Selecting previously unselected package pv.
(Reading database ... 126315 files and directories currently installed.)
Preparing to unpack .../pv_1.6.6-1build2_amd64.deb ...
Unpacking pv (1.6.6-1build2) ...
Selecting previously unselected package tree.
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Setting up pv (1.6.6-1build2) ...
Processing triggers for man-db (2.10.2-1) ...

In [ ]:
# prompt: # prompt: load the data pytorch training with test train val splits
# # we have images in ./images forlder and
# # mpii_human_pose_v1_u12_2
# # ├── bsd.txt
# # ├── mpii_human_pose_v1_u12_1.mat
# # └── README.md

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import scipy.io as sio
import os
from PIL import Image
import numpy as np
import PIL

# Define the custom dataset class
class MPIIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format

        if self.transform:
            image = self.transform(image)

        # Return the image and label (adjust label processing as needed)
        return image, label

# Load the annotation data
mat_data = sio.loadmat('mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat')
annotations = mat_data['RELEASE']

# Create a list to store the dataset (image path, label) pairs
dataset = []

for i in range(len(annotations['annolist'][0])):
    # More robust image name extraction
    image_data = annotations['annolist'][0][i]['image'][0][0]
    if 'name' in image_data.dtype.names:
        image_name = str(image_data['name'][0][0])
    else:  # Handle cases where 'name' is not directly present
        image_name = str(image_data[0][0])

    image_path = os.path.join('mpii_human_pose_v1_u12_2/images', image_name) #Corrected image path

    # Extract labels, handling cases where 'annorect' is missing
    # Check if 'annorect' is a field in the current annotation
    label = annotations['annolist'][0][i]['annorect'][0] if 'annorect' in annotations['annolist'][0][i].dtype.names else None
    #if label is not None and 'objpos' in label.dtype.names: #Added extra condtion to check objpos is present in label
    #    label = label['objpos'][0] if 'objpos' in label.dtype.names else None
    #else:
    #    label = None

    # Check if the image file actually exists
    if os.path.exists(image_path):
        dataset.append((image_path, label))
    else:
        print(f"Warning: Image not found: {image_path}")


# Split the dataset into train, validation, and test sets
# You can adjust the split ratios as needed
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(dataset) * train_ratio)
val_size = int(len(dataset) * val_ratio)
test_size = len(dataset) - train_size - val_size

train_dataset = dataset[:train_size]
val_dataset = dataset[train_size:train_size + val_size]
test_dataset = dataset[train_size + val_size:]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

transform = transforms.Compose([
    transforms.Resize(256),  # Resize the images
    transforms.CenterCrop(224),  # Crop the images
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

train_dataset = MPIIDataset(train_dataset, transform=transform)
val_dataset = MPIIDataset(val_dataset, transform=transform)
test_dataset = MPIIDataset(test_dataset, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Train dataset size: 0
Validation dataset size: 0
Test dataset size: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
mat_data = sio.loadmat('mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat')
annotations = mat_data['RELEASE']
annolist = annotations['annolist']

for annotation in annolist:
  print(annotation)
  break

#for i in range(len(annotations['annolist'][0])):


[array([[(array([[(array(['037454012.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]), array([[(array([[3.88073395]]), array([[(array([[601]], dtype=uint16), array([[380]], dtype=uint16))]],
                       dtype=[('x', 'O'), ('y', 'O')]))                                                         ]],
               dtype=[('scale', 'O'), ('objpos', 'O')]), array([], shape=(1, 0), dtype=float64), array([], shape=(1, 0), dtype=float64))                                                      ,
         (array([[(array(['095071431.jpg'], dtype='<U13'),)]], dtype=[('name', 'O')]), array([[(array([[8.07816613]]), array([[(array([[881]], dtype=uint16), array([[394]], dtype=uint16))]],
                       dtype=[('x', 'O'), ('y', 'O')]))                                                         ,
                 (array([[8.90412938]]), array([[(array([[338]], dtype=uint16), array([[210]], dtype=uint8))]],
                       dtype=[('x', 'O'), ('y', 'O')]))                             

In [ ]:
!pip install scipy

In [14]:
# Data annotations described here:
# https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-dataset/download

import scipy.io
import numpy as np

mat_file_path = 'mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'
release_data = scipy.io.loadmat(mat_file_path)['RELEASE']
print(f"Fields inside 'RELEASE' structured array: {release_data.dtype.names}")
annolist = release_data['annolist'][0, 0]
print(f"Shape of 'annolist': {annolist.shape}")


first_image_annots = annolist[0, 0] # Or maybe annolist[0] depending on shape
print("\nInspecti----", first_image_annots.dtype.names)
print(first_image_annots['image'][0][0][0][0])

act = release_data['act'][0, 0]

print(f"act ::: {act}")


for i in range(annolist.shape[1]):  # Iterate through the second dimension of annolist
    image_annot = annolist[0, i]  # Access the element using index i in the second dimension
    filename = image_annot['image'][0][0][0][0]  # Extract the filename

    print(f"Filename for index {i}: {filename}")
    if i > 5:
      print("Breaking", i)
      break



Fields inside 'RELEASE' structured array: ('annolist', 'img_train', 'version', 'single_person', 'act', 'video_list')
Shape of 'annolist': (1, 24987)

Inspecti---- ('image', 'annorect', 'frame_sec', 'vididx')
037454012.jpg
act ::: [[(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 ...
 [(array(['transportation'], dtype='<U14'), array(['pushing car'], dtype='<U11'), array([[972]], dtype=uint16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]]
Filename for index 0: 037454012.jpg
Filename for index 1: 095071431.jpg
Filename for index 2: 073199394.jpg
Filename for index 3: 059865848.jpg
Filename for index 4: 015601864.jpg
Filename for index 5: 015599452.jpg
Filename for index 6: 005808361.jpg

In [23]:
act = release_data['act'][0, 0]
data_list = []

for i, item in enumerate(act):
    print(f"Item {i}: {item}") # Print the entire item for inspection
    print(f"Item {i} shape: {item.shape}") # Print item shape
    print(f"Item {i} dtype: {item.dtype}") # Print item dtype

    # Attempt to extract values based on observed structure
    # Adjust these lines based on the output of the print statements
    try:
        act_name = item[0][0][0] if len(item[0]) > 0 and len(item[0][0]) > 0 else None
        cat_name = item[0][1][0] if len(item[0]) > 1 and len(item[0][1]) > 0 else None
        act_id = item[0][2][0][0] if len(item[0]) > 2 and len(item[0][2]) > 0 else None
    except IndexError:
        print("IndexError encountered. Skipping this item.")
        act_name, cat_name, act_id = None, None, None

    data_list.append((i, act_name, cat_name, act_id))

    if i > 50:  # Limit output for inspection
        break

# Print the extracted data
for i, act_name, cat_name, act_id in data_list:
    print(f"I: {i} Act Name: {act_name}, Cat Name: {cat_name}, Act ID: {act_id}")
    #print(f"I: {i} Act Name: {type(act_name)}, Cat Name: {type(cat_name)}, Act ID: {type(act_id)}")

Item 0: [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
Item 0 shape: (1,)
Item 0 dtype: [('cat_name', 'O'), ('act_name', 'O'), ('act_id', 'O')]
Item 1: [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
Item 1 shape: (1,)
Item 1 dtype: [('cat_name', 'O'), ('act_name', 'O'), ('act_id', 'O')]
Item 2: [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
Item 2 shape: (1,)
Item 2 dtype: [('cat_name', 'O'), ('act_name', 'O'), ('act_id', 'O')]
Item 3: [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
Item 3 shape: (1,)
Item 3 dtype: [('cat_name', 'O'), ('act_name', 'O'), ('act_id', 'O')]
Item 4: [(array(['sports'], dtype='<U6'), array(['curling'], dtype='<U7'), array([[1]], dtype=uint8))]
Item 4 shape: (1,)
Item 4 dtype: [('cat_name', 'O'), ('act_name', 'O'), ('act_id', 'O')]
Item 5: [(array(['sports'], dtype='<U6'), array(['curling'], dtype='<U7'), array([[1]], dtype=uint8))]
I